# **Project 1**

# Loading data, importing libraries

In [229]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import optimize
from numpy import random
import random
import math
n = 1000
url = 'https://raw.githubusercontent.com/anonymous-student123/bid_data/main/bid_data.csv'
df = pd.read_csv(url)

Calculating probabilities with uniform distrubtion


In [230]:
values = np.array([10*v for v in range(1,11)])
c_bids = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45, 50])
a_bids = np.array([0.01, 10.01, 20.01, 30.01, 40.01, 50.01, 60.01, 70.01, 70.02, 80.02])
test_bids = [0 for i in range(10)]

# removing our bids
df_c = df.drop(index=1)
df_a = df.drop(index=37)

def calculate_prob_and_utility(p,x=None,i=None, sample = None, mc_bids = None):
  probability_arr_c = []
  probability_arr_a = []
  probability_arr_test = []
  probability_arr_MC = []

  if p == 'c':
    df_s = df_c
    own_bids = c_bids
    probability_arr = probability_arr_c
  if p == 'a':
    df_s = df_a
    own_bids = a_bids
    probability_arr = probability_arr_a
  if p == 'test':
    df_s = df
    own_bids = test_bids
    own_bids[i] = x
    probability_arr = probability_arr_test
  if p == 'algorithm':
    df_s = sample
    own_bids = test_bids
    own_bids[i] = x
    probability_arr = probability_arr_test
  if p == 'algorithm-monte-carlo':
    df_s = df
    own_bids = mc_bids
    probability_arr = probability_arr_MC

  # variable for potentially won bets
  total_won = 0

  for i in range(10):
    # our bid
    bid = own_bids[i]

    # DF of bools, true being bids we win against
    win = (df_s < bid)

    # DF of bools, true being bids we tie against
    tie = (df_s == bid)

    # now we simply divide favorable outcomes by total outcomes because the
    # distribution is uniform: any other student's value and bid is equally likely
    # note that we halve the number of "favorable" events in the case of ties
    # to account for the coin flip deciding the outcome.
    favorable = win.to_numpy().sum() + 0.5 * tie.to_numpy().sum()
    probability = favorable / win.size

    total_won += favorable

    # array for probabilities
    probability_arr.append(probability)

  # now we just take the dot product of each probability of win
  prob_win = total_won / (df_s.size * 10)
  utility = np.dot(np.array(probability_arr), values - own_bids) / 10
  return prob_win, utility, probability_arr, np.multiply(np.array(probability_arr), values - own_bids)

prob_win_c, utility_c, prob_arr_c, utility_per_value_c = calculate_prob_and_utility('c')
prob_win_a, utility_a, prob_arr_a, utility_per_value_a = calculate_prob_and_utility('a')

print("Probability C overall wins a bet: ", prob_win_c)
print("C's expected utility: ", utility_c)
print("C's bid winning probability per value: ", prob_arr_c)
print("C's expected utility per value: ", utility_per_value_c)
print("------------")
print("Probability A overall wins a bet: ", prob_win_a)
print("A's expected utility", utility_a)
print("A's bid winning probability per value: ", prob_arr_a)
print("A's expected utility per value: ", utility_per_value_a)

Probability C overall wins a bet:  0.3464634146341463
C's expected utility:  12.240853658536585
C's bid winning probability per value:  [0.025609756097560974, 0.10975609756097561, 0.20243902439024392, 0.2548780487804878, 0.3280487804878049, 0.37926829268292683, 0.45365853658536587, 0.5060975609756098, 0.5792682926829268, 0.625609756097561]
C's expected utility per value:  [ 0.12804878  1.09756098  3.03658537  5.09756098  8.20121951 11.37804878
 15.87804878 20.24390244 26.06707317 31.2804878 ]
------------
Probability A overall wins a bet:  0.5541463414634147
A's expected utility 7.31219268292683
A's bid winning probability per value:  [0.00853658536585366, 0.16219512195121952, 0.2939024390243902, 0.42073170731707316, 0.55, 0.6695121951219513, 0.7987804878048781, 0.8597560975609756, 0.8621951219512195, 0.9158536585365854]
A's expected utility per value:  [ 0.08528049  1.62032927  2.93608537  4.20310976  5.4945      6.68842683
  7.97981707  8.58896341 17.22665854 18.2987561 ]


Calculating probabilities with Monte-Carlo

In [231]:
c_bids = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45, 50])
a_bids = np.array([0.01, 10.01, 20.01, 30.01, 40.01, 50.01, 60.01, 70.01, 70.02, 80.02])
test_bids = [0 for i in range(10)]
# run_auction takes in a number of auctions to run and a player
# then outputs the calculated percentage of auctions won and
# the average utility over these auctions
def run_auction(n, p, x=None, i=None):
  if p == 'c':
    df_s = df_c
    own_bids = c_bids
  if p == 'a':
    df_s = df_a
    own_bids = a_bids
  if p == 'test':
    # this other "player" is used while optimizing later: it simulates an auction
    # with the only bid being x given value i. this way we can calculate
    # maximum expectation componentwise
    df_s = df
    own_bids = test_bids
    own_bids[i] = x

  wins = 0
  utility = 0
  auctions = 0
  utility_index_i = np.array([0 for i in range(10)])
  auctions_index_i = np.array([0 for i in range(10)])
  wins_index_i = np.array([0 for i in range(10)])

  # looping over each possible value [1,..,10] (in other words [10, ..., 100])
  for i in range(n):
    # generating a random value for self
    self_rn = random.randrange(0,10)
    self_value = (self_rn+1)*10
    self_bid = own_bids[self_rn]


    # generating a random value for other student
    # then selecting this student with sample()
    other_rn = random.randrange(0,10)
    other_value = (other_rn+1) * 10
    random_bid_given_value = df_c.sample()
    other_bid = random_bid_given_value.iat[0,other_rn]


    # generating random value for coin
    coin = random.randrange(0,101) % 2
    
    # the below increments stats given the results of the simulated
    # auction. if a bid is tied it uses a coin RV to update the 
    # counts
    auctions += 1
    auctions_index_i[self_rn] += 1

    if self_bid > other_bid:
      wins += 1
      wins_index_i[self_rn] += 1

      utility += (self_value - self_bid)
      utility_index_i[self_rn] += (self_value - self_bid)
    
    if self_bid == other_bid:
      if coin == 1:
        wins += 1
        wins_index_i[self_rn] += 1

        utility += (self_value - self_bid)
        utility_index_i[self_rn] += (self_value - self_bid)

  return wins / auctions, utility / auctions, np.divide(wins_index_i, auctions_index_i), np.divide(utility_index_i, auctions_index_i)

c_win_percent, c_avg_utility, c_win_percent_indices, c_avg_utility_indices = run_auction(n, 'c')
a_win_percent, a_avg_utility, a_win_percent_indices, a_avg_utility_indices = run_auction(n, 'a')
print("C's simulated win % with ", n, " auctions: ", c_win_percent)
print("C's simulated average utility with ", n, " auctions: ", c_avg_utility)
print("C's bid winning probability per value: ", c_win_percent_indices)
print("C's expected utility per value: ", c_avg_utility_indices)
print("------")
print("A's simulated win % with ", n, " auctions: ", a_win_percent)
print("A's simulated average utility with ", n, " auctions: ", a_avg_utility)
print("A's bid winning probability per value with", n, "auctions: ", a_win_percent_indices)
print("A's expected utility per value with ", n, "auctions: ", a_avg_utility_indices)

C's simulated win % with  1000  auctions:  0.348
C's simulated average utility with  1000  auctions:  12.48
C's bid winning probability per value:  [0.02061856 0.06521739 0.22222222 0.23762376 0.38383838 0.45918367
 0.40625    0.44660194 0.47413793 0.67592593]
C's expected utility per value:  [ 0.10309278  0.65217391  3.33333333  4.75247525  9.5959596  13.7755102
 14.21875    17.86407767 21.3362069  33.7962963 ]
------
A's simulated win % with  1000  auctions:  0.558
A's simulated average utility with  1000  auctions:  7.2527399999999105
A's bid winning probability per value with 1000 auctions:  [0.00840336 0.21698113 0.3627451  0.43396226 0.69512195 0.625
 0.7979798  0.86792453 0.81052632 0.93814433]
A's expected utility per value with  1000 auctions:  [ 0.07563025  1.95283019  3.26470588  3.90566038  6.25609756  5.625
  7.18181818  7.81132075 15.4        17.82474227]


Error bounds

In [232]:
print("DIFFERENCES BETWEEN SIMULATED AND CALCULATED STATISTICS: ")
print("C's agregate probability difference: ", np.abs(c_win_percent - prob_win_c))
print("C's agregate expected utility difference: ", np.abs(c_avg_utility - utility_c))
print("C's componentwise probability difference: ", np.abs(prob_arr_c - c_win_percent_indices))
print("C's componentwise expected utility difference: ", np.abs(utility_per_value_c - c_avg_utility_indices))
print("-------")
print("A's agregate probability difference: ", np.abs(a_win_percent - prob_win_a))
print("A's agregate expected utility difference: ", np.abs(a_avg_utility - utility_a))
print("A's componentwise probability difference: ", np.abs(prob_arr_a - a_win_percent_indices))
print("A's componentwise expected utility difference: ", np.abs(utility_per_value_a - a_avg_utility_indices))

DIFFERENCES BETWEEN SIMULATED AND CALCULATED STATISTICS: 
C's agregate probability difference:  0.00153658536585366
C's agregate expected utility difference:  0.23914634146341562
C's componentwise probability difference:  [0.0049912  0.04453871 0.0197832  0.01725429 0.0557896  0.07991538
 0.04740854 0.05949562 0.10513036 0.05031617]
C's componentwise expected utility difference:  [0.024956   0.44538706 0.29674797 0.34508573 1.39474008 2.39746142
 1.65929878 2.37982477 4.73086627 2.51580849]
-------
A's agregate probability difference:  0.0038536585365853693
A's agregate expected utility difference:  0.05945268292691974
A's componentwise probability difference:  [1.33224021e-04 5.47860101e-02 6.88426590e-02 1.32305568e-02
 1.45121951e-01 4.45121951e-02 8.00689825e-04 8.16843074e-03
 5.16688062e-02 2.22906714e-02]
A's componentwise expected utility difference:  [0.00965024 0.33250092 0.32862052 0.29744938 0.76159756 1.06342683
 0.79799889 0.77764266 1.82665854 0.47401383]


Finding optimal bid to maximize utility

In [233]:
epsilon = 0.00001

def utility_coord_i(x, i, sample=None):
  if sample is not None:
    return calculate_prob_and_utility('test', x, i, sample)[1]
  return calculate_prob_and_utility('test', x, i)[1]

def utility_monte_carlo_coord_i(x, i, n):
  return run_auction(n, 'test', x, i)[1]

def optimal_bid_one_index(index, df_g=None):
  utilities = []
  if df_g is not None:
    df_s = df_g
    values_arrays = df_s.to_numpy()
    all_values = np.sort(np.unique(np.concatenate(values_arrays)))
  else:
    values_arrays = df.to_numpy()
    all_values = np.sort(np.unique(np.concatenate(values_arrays)))
  for val in all_values:
    if (index+1) * 10 < val:
      if not utilities:
        utilities.append([0,0])
      break
    if df_g is not None:
      utilities.append([val, utility_coord_i(val,index, df_g)*10])
      utilities.append([val + epsilon, utility_coord_i(val + epsilon,index, df_g)*10])
    else:
      utilities.append([val, utility_coord_i(val,index)*10])
      utilities.append([val + epsilon, utility_coord_i(val + epsilon,index)*10])
  return max(utilities, key= lambda x: x[1])

def monte_carlo_bid_one_index(index):
  utilities = []
  values_arrays = df.to_numpy()
  all_values = np.sort(np.unique(np.concatenate(values_arrays)))
  for val in all_values:
    if (index+1) * 10 < val:
      break
    utilities.append([val, utility_monte_carlo_coord_i(val,index, n)])
    utilities.append([val + epsilon, utility_monte_carlo_coord_i(val + epsilon,index, n)])
  return max(utilities, key= lambda x: x[1])


In [234]:
best_utilities_arr = np.array([optimal_bid_one_index(i) for i in range(10)])
optimal = sum(j for i,j in best_utilities_arr)
print("The best bids and expected utilities for each are given in the following ",
      "array of the form [bid, expected utility].")
print(best_utilities_arr)
print("By linearity of expectation we can sum over these and find that our",
      "optimal aggregate expected utility is: ", optimal/10)

best_monte_carlo_utilities_arr = np.array([monte_carlo_bid_one_index(i) for i in range(10)])
mc_optimal = sum(j for i,j in best_monte_carlo_utilities_arr)
print("The best bids and expected utilities via Monte Carlo for each are given in the following ",
      "array of the form [bid, expected utility].")
print(best_monte_carlo_utilities_arr)
print("By linearity of expectation we can sum over these and find that our",
      "optimal aggregate expected utility is: ", mc_optimal)

The best bids and expected utilities for each are given in the following  array of the form [bid, expected utility].
[[ 5.00001     0.84523769]
 [11.00001     2.33332988]
 [11.00001     4.20237464]
 [21.00001     6.59760733]
 [21.00001     9.72855431]
 [31.00001    13.31664169]
 [31.00001    17.68568143]
 [41.00001    22.60471926]
 [41.00001    28.23566257]
 [51.00001    33.9499375 ]]
By linearity of expectation we can sum over these and find that our optimal aggregate expected utility is:  13.949974630952374
The best bids and expected utilities via Monte Carlo for each are given in the following  array of the form [bid, expected utility].
[[5.00001000e+00 3.99937200e-02]
 [1.00000100e+01 2.29993650e-01]
 [1.00100100e+01 5.79703760e-01]
 [1.50000100e+01 7.49993810e-01]
 [2.82000100e+01 1.15539354e+00]
 [3.00100100e+01 1.61945340e+00]
 [4.10000100e+01 2.29099366e+00]
 [3.60000000e+01 2.68399449e+00]
 [3.50000100e+01 3.62999374e+00]
 [4.99000000e+01 4.05809453e+00]]
By linearity of expec

Error with simulated

In [238]:
best_monte_carlo_utilities_arr - best_utilities_arr

array([[  0.        ,  -0.80524397],
       [ -1.        ,  -2.10333623],
       [ -0.99      ,  -3.62267088],
       [ -6.        ,  -5.84761352],
       [  7.2       ,  -8.57316077],
       [ -0.99      , -11.69718829],
       [ 10.        , -15.39468777],
       [ -5.00001   , -19.92072477],
       [ -6.        , -24.60566883],
       [ -1.10001   , -29.89184297]])

Algorithm

In [236]:
def find_optimal_bid(n):
  if n > len(df):
    n = len(df)
  df_sample = df.sample(n)
  # this simply calculates the optimal bid, as done above, but given
  # only a select n number of bids from the original dataset
  # (can be adapted to input custom bids, but functions equivalently by
  # picking this random sample. this is also easier to test using monte-carlo)
  return np.array([optimal_bid_one_index(i, df_sample)[0] for i in range(10)])

find_optimal_bid(1)

array([1.100000e-04, 1.000011e+01, 1.000011e+01, 2.000011e+01,
       2.000011e+01, 3.000011e+01, 3.000011e+01, 4.000011e+01,
       4.000011e+01, 5.000011e+01])

Monte Carlo on Algorithm

In [237]:
def monte_carlo_algorithm(N, s):
  # N: number of times to run monte-carlo
  # s: sample size
  agg_utility = 0
  for i in range(s):
    bids = find_optimal_bid(s)
    compared_to_real = calculate_prob_and_utility('algorithm-monte-carlo', mc_bids = bids)
    agg_utility += compared_to_real[1]
  return agg_utility/s

one_sample_average_utility = monte_carlo_algorithm(n, 1)
ten_sample_average_utility = monte_carlo_algorithm(n, 10)
full_sample_average_utility = monte_carlo_algorithm(n, 100)

print("given ", n, "samples:")
print("when N=1 is given, our algorithm has average utility ", one_sample_average_utility)
print("when N=10 is given, our algorithm has average utility ", ten_sample_average_utility)
print("when N=FULL SET is given, our algorithm has average utility ", full_sample_average_utility)

given  1000 samples:
when N=1 is given, our algorithm has average utility  12.976186869047618
when N=10 is given, our algorithm has average utility  13.527770038571429
when N=FULL SET is given, our algorithm has average utility  13.557139066666656
